### National Transportation Safety Board Accident/Incident Data

In [47]:
import pandas as pd
import xml.etree.ElementTree as ET  #XML parsing (https://docs.python.org/2/library/xml.etree.elementtree.html)
import numpy as np
import re #regex
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_columns', None)

In [48]:
#Load data
xmltree = ET.parse('AviationData.xml')

data = []
for el in xmltree.iterfind('./*'):
    for i in el.iterfind('*'):
        data.append(dict(i.items()))

#Convert list to dataframe
df = pd.DataFrame(data)


In [49]:
df.head()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,EventDate,EventId,FARDescription,InjurySeverity,InvestigationType,Latitude,Location,Longitude,Make,Model,NumberOfEngines,PublicationDate,PurposeOfFlight,RegistrationNumber,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition
0,GAA15CA244,,,,,,,,United States,,09/01/2015,20150901X74304,,,Accident,,"Truckee, CA",,JOE SALOMONE,SUPER CUB SQ2,,,,N786AB,Preliminary,,,,,,
1,CEN15LA392,,Airplane,Substantial,IWS,WEST HOUSTON,No,LANDING,United States,,08/31/2015,20150901X92332,Part 91: General Aviation,Non-Fatal,Accident,29.809444,"Houston, TX",-95.668889,CESSNA,T240,,09/04/2015,Instructional,N452CS,Preliminary,,,,,2,VMC
2,GAA15CA246,,,,,,,,United States,,08/30/2015,20150902X61853,,,Accident,,"New Milford, CT",,CESSNA,170A,,,,N112RP,Preliminary,,,,,,
3,GAA15CA243,,,,,,,,United States,,08/30/2015,20150901X02521,,,Accident,,"New Enterprise, PA",,CESSNA,172C,,,,N8404X,Preliminary,,,,,,
4,GAA15CA242,,,,,,,,United States,,08/29/2015,20150831X61356,,,Accident,,"MIDLOTHIAN, TX",,CESSNA,305A,,,,N5255G,Preliminary,,,,,,


In [20]:
tree.getroot().tag

'{http://www.ntsb.gov}DATA'

In [19]:
tree.getroot().attrib

{}

In [62]:
def clean_xml_data(df):
    
    #Subset to US fatalities, since that is the primary interest of the NTSB data
    df_subset = df[df['Country'] == 'United States']
    
    #Replace blank string values with NaN
    df_copy = df_subset.replace('', np.nan)
    
    #Clean AirCarrier Names
    df_copy['AirCarrier'] = df_subset['AirCarrier'].str.title()  #convert to lowercase to deal with companies captalized differently
    #TODO: Use "DBA" name if available
    
    #Split States/Cities in Location Field
    df_copy['City'] = df_subset.Location.str.title().str.split(', ', expand=True)[0]
    df_copy['State'] = df_subset.Location.str.upper().str.split(', ', expand=True)[1]
    
    #Strip Number from Injury Serverity Field
    df_copy['fatalities'] = df_subset['InjurySeverity'].str.extract(r'(\d+)')
    
    #Extract variables from date field
    df_copy['year'] = pd.DatetimeIndex(df_subset['EventDate']).year
    df_copy['month'] = pd.DatetimeIndex(df_subset['EventDate']).month
    df_copy['EventDate'] = pd.to_datetime(df_subset['EventDate'])
    
    
    return(df_copy)
    
df_clean = clean_xml_data(df)
df_clean.head()

df_clean.to_csv('aviation_data.csv')

In [63]:
print('There are {} incidents in the data, with {} columns. The columns available are: {}'.format(df.shape[0], df.shape[1], df.columns.to_list()))

There are 77257 incidents in the data, with 32 columns. The columns available are: ['AccidentNumber', 'AirCarrier', 'AircraftCategory', 'AircraftDamage', 'AirportCode', 'AirportName', 'AmateurBuilt', 'BroadPhaseOfFlight', 'Country', 'EngineType', 'EventDate', 'EventId', 'FARDescription', 'InjurySeverity', 'InvestigationType', 'Latitude', 'Location', 'Longitude', 'Make', 'Model', 'NumberOfEngines', 'PublicationDate', 'PurposeOfFlight', 'RegistrationNumber', 'ReportStatus', 'Schedule', 'TotalFatalInjuries', 'TotalMinorInjuries', 'TotalSeriousInjuries', 'TotalUninjured', 'WeatherCondition', 'year']


In [64]:
#Count missing values 
df_clean.isna().sum()

AccidentNumber              0
AirCarrier                  0
AircraftCategory        58199
AircraftDamage           1856
AirportCode             30266
AirportName             26618
AmateurBuilt              273
BroadPhaseOfFlight       2815
Country                     0
EngineType               1143
EventDate                   2
EventId                     0
FARDescription          58053
InjurySeverity            106
InvestigationType           0
Latitude                50507
Location                   33
Longitude               50517
Make                       23
Model                      40
NumberOfEngines          1507
PublicationDate         12462
PurposeOfFlight          2001
RegistrationNumber        148
ReportStatus                0
Schedule                63612
TotalFatalInjuries      19945
TotalMinorInjuries      19904
TotalSeriousInjuries    21026
TotalUninjured           9389
WeatherCondition          373
year                        2
City                        0
State     

In [136]:
df_clean.describe()

,AccidentNumber,AirCarrier,AircraftCategory,AircraftDamage,AirportCode,AirportName,AmateurBuilt,BroadPhaseOfFlight,Country,EngineType,...,RegistrationNumber,ReportStatus,Schedule,TotalFatalInjuries,TotalMinorInjuries,TotalSeriousInjuries,TotalUninjured,WeatherCondition,City,State
count,77257,77257,16520,74873,43477,47331,76603,71030,76747,74048,...,74501,77257,11379,55791,54733,53744,65862,75268,77257,76671
unique,77257,2720,12,3,9488,22283,2,12,173,14,...,67492,4,3,117,62,40,363,3,14542,329
top,LAX86LA318,,Airplane,Substantial,NONE,N/A,No,LANDING,United States,Reciprocating,...,NONE,Probable Cause,UNK,0,0,0,1,VMC,Anchorage,CA
freq,1,73439,14210,55420,1457,1210,69198,18553,73076,63016,...,363,72264,4099,40363,40342,42955,22029,68764,501,8027


In [65]:
df_clean['EventDate'].describe()

count                   73074
unique                  12089
top       1984-06-30 00:00:00
freq                       25
first     1948-10-24 00:00:00
last      2015-09-01 00:00:00
Name: EventDate, dtype: object

In [63]:
for col in df.columns:
    
    print(df_clean[col].value_counts()[0:20])

LAX86LA318     1
SEA02LA170     1
CEN13CA144     1
CEN15LA233     1
BFO93LA086     1
LAX03LA112     1
DEN00FA086     1
NYC91LA054     1
LAX00LA167     1
ANC98LA130     1
MKC90LA021     1
LAX98LA302A    1
FTW02LA074     1
SEA05LA065     1
ERA12FA175     1
ATL83LA162     1
FTW89LA056     1
CHI93LA064     1
ERA10LA184     1
LAX91FA344     1
Name: AccidentNumber, dtype: int64
UNITED AIRLINES                           49
AMERICAN AIRLINES                         41
CONTINENTAL AIRLINES                      25
USAIR                                     24
DELTA AIR LINES INC                       23
AMERICAN AIRLINES, INC.                   22
SOUTHWEST AIRLINES CO                     21
CONTINENTAL AIRLINES, INC.                19
UNITED AIR LINES INC                      14
AMERICAN AIRLINES INC                     12
SIMMONS AIRLINES (DBA: AMERICAN EAGLE)    12
US AIRWAYS INC                            12
TRANS WORLD AIRLINES                      11
DELTA AIRLINES                          

In [ ]:
#Plot number of incidents over time

